In [ ]:
import pickle
import numpy as np
import plotly.graph_objects as go

import yaml
from scipy.linalg import expm, inv
from numpy import dot, eye
import sys
import torch
import numpy as np

In [ ]:
eef_poses = [
    [3.008, -0.668, 0.549, -0.16574, -0.268, 0.5065], # right back 0
    [3.008, -0.5914, 0.633, -0.111, -0.2634, 0.485], # right back 1
    [-2.31092, -0.24056, -0.99504, -0.21117, -0.12188, 0.41125], #right back 2
    [-2.51890, -0.54126, -0.08807, -0.00074, -0.49342, 0.42817], # right back corner
    [-3.008, -0.668, -0.549, -0.16574, 0.268, 0.5065], # left back 0
    [-3.008, -0.5914, -0.633, -0.111, 0.2634, 0.485], # left back 1
    [-3.008, -0.5914, -0.633, -0.061, 0.2634, 0.485], # left back 1 x + 5cm
    [-3.008, -0.5914, -0.633, -0.011, 0.2634, 0.485], # left back 1 x + 10cm
    [2.31092, -0.24056, 0.99504, -0.21117, 0.12188, 0.41125], # left back 2
    [2.39645, -0.54843, 0.08445, -0.00101, 0.49278, 0.42958], # left back corner
]

In [ ]:
from numpy import dot, eye, zeros, outer
from numpy.linalg import inv

def log(R):
    # Rotation matrix logarithm
    theta = np.arccos((R[0,0] + R[1,1] + R[2,2] - 1.0)/2.0)
    return np.array([R[2,1] - R[1,2], 
                     R[0,2] - R[2,0], 
                     R[1,0] - R[0,1]]) * theta / (2*np.sin(theta))

def invsqrt(mat):
    u,s,v = np.linalg.svd(mat)
    return u.dot(np.diag(1.0/np.sqrt(s))).dot(v)

def calibrate(A, B):
    #transform pairs A_i, B_i
    N = len(A)
    M = np.zeros((3,3))
    for i in range(N):
        Ra, Rb = A[i][0:3, 0:3], B[i][0:3, 0:3]
        M += outer(log(Rb), log(Ra))

    Rx = dot(invsqrt(dot(M.T, M)), M.T)

    C = zeros((3*N, 3))
    d = zeros((3*N, 1))
    for i in range(N):
        Ra,ta = A[i][0:3, 0:3], A[i][0:3, 3]
        Rb,tb = B[i][0:3, 0:3], B[i][0:3, 3]
        C[3*i:3*i+3, :] = eye(3) - Ra
        d[3*i:3*i+3, 0] = ta - dot(Rx, tb)

    tx = dot(inv(dot(C.T, C)), dot(C.T, d))
    return Rx, tx.flatten()

def euler_to_rotation_matrix(roll, pitch, yaw):
    """
    Convert euler angles (roll, pitch, yaw) to a rotation matrix.
    """
    R_x = torch.tensor([[1, 0, 0],
                        [0, torch.cos(roll), -torch.sin(roll)],
                        [0, torch.sin(roll), torch.cos(roll)]])
    
    R_y = torch.tensor([[torch.cos(pitch), 0, torch.sin(pitch)],
                        [0, 1, 0],
                        [-torch.sin(pitch), 0, torch.cos(pitch)]])
    
    R_z = torch.tensor([[torch.cos(yaw), -torch.sin(yaw), 0],
                        [torch.sin(yaw), torch.cos(yaw), 0],
                        [0, 0, 1]])
    
    R = torch.matmul(R_z, torch.matmul(R_y, R_x))
    return R
    
def pose_to_transform(batch):
    """
    Construct a batch of transformation matrices from a batch of roll, pitch, yaw, x, y, z.
    The input batch should have the shape (n, 6), where each row is (roll, pitch, yaw, x, y, z).
    The output will have the shape (n, 4, 4), representing the transformation matrices.
    """
    n = batch.size(0)
    transformation_matrices = torch.zeros((n, 4, 4))
    
    for i in range(n):
        roll, pitch, yaw, x, y, z = batch[i]
        rotation_matrix = euler_to_rotation_matrix(roll, pitch, yaw)
        transformation_matrix = torch.eye(4)
        transformation_matrix[:3, :3] = rotation_matrix
        transformation_matrix[:3, 3] = torch.tensor([x, y, z])
        transformation_matrices[i] = transformation_matrix
    
    return transformation_matrices

In [ ]:
eff_poses_tor_o=pose_to_transform(torch.tensor(eef_poses))
im_poses_o=torch.load("im_poses_not_inv.tar").cpu().detach()

useful_indices = [0,1,2,3,4,7,8,9]

eff_poses_tor=eff_poses_tor_o[useful_indices]
im_poses=im_poses_o[useful_indices]

In [ ]:
def compute_A_B_Rx(eff_poses_tor,im_poses):
    A, B = [], []
    for i in range(1,len(im_poses)):
        p = eff_poses_tor[i-1], im_poses[i-1]
        n = eff_poses_tor[i], im_poses[i]
        A.append(dot(inv(p[0]), n[0]))
        B.append(dot(inv(p[1]), n[1]))

    N = len(A)
    M = np.zeros((3,3))
    for i in range(N):
        Ra, Rb = A[i][0:3, 0:3], B[i][0:3, 0:3]
        M += outer(log(Rb), log(Ra))

    Rx = dot(invsqrt(dot(M.T, M)), M.T)
    return(A, B, Rx, N)

In [ ]:
def compute_cost(scale, A, B, Rx, N):
    C = zeros((3*N, 3))
    d = zeros((3*N, 1))
    for i in range(N):
        Ra,ta = A[i][0:3, 0:3], A[i][0:3, 3]
        Rb,tb = B[i][0:3, 0:3], B[i][0:3, 3]
        C[3*i:3*i+3, :] = eye(3) - Ra
        d[3*i:3*i+3, 0] = ta - dot(Rx, scale*tb)

    tx = dot(inv(dot(C.T, C)), dot(C.T, d))
    J = np.sum(np.linalg.norm(np.dot(C, tx) - d)**2)
    return(J, tx)

In [ ]:
vals_sc=np.linspace(1,4,20000).reshape(-1)
A, B, Rx, N=compute_A_B_Rx(eff_poses_tor,im_poses)

In [ ]:
%%time
#we can just brute force
costs_list=[]
tx_list=[]
for i in range(len(vals_sc)):
    J_res, tx_res=compute_cost(vals_sc[i],A, B, Rx, N)
    costs_list.append(J_res)
    tx_list.append(tx_res)
costs_tor=torch.tensor(costs_list)

In [ ]:
min_inds=torch.argmin(costs_tor)
scale=vals_sc[min_inds]
print("scale found: {}".format(scale))

In [ ]:
#load poses again and rescale
im_poses_o=torch.load("im_poses_not_inv.tar").cpu().detach()
im_poses_o[:,:3,3]=im_poses_o[:,:3,3]*scale
im_poses=im_poses_o[useful_indices]
A, B, Rx, N=compute_A_B_Rx(eff_poses_tor,im_poses)
J,tx=compute_cost(1.,A, B, Rx, N) # the scale factor is just 1 as we have already rescaled

In [ ]:
X = eye(4)
X[0:3, 0:3] = Rx
X[0:3, -1] = tx.reshape(-1)

tmp_list=[]
for i in range(len(im_poses)):
    rob = eff_poses_tor[i]
    obj = im_poses[i]
    tmp = dot(rob, dot(X, inv(obj)))
    tmp_list.append(tmp)
tmp_tor=torch.tensor(tmp_list)
world_pose=tmp_tor.mean(dim=0)
c_pos=torch.tensor(world_pose).float()@(im_poses_o)
c_pos_n=c_pos[:,:3,3]
eff_poses_n=eff_poses_tor_o[:,:3,3]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=c_pos_n[:, 0].detach().cpu(),  # X coordinates
    y=c_pos_n[:, 1].detach().cpu(),  # Y coordinates
    z=c_pos_n[:, 2].detach().cpu(),  # Z coordinates
                           mode='markers', marker=dict(size=5, color='red'),name="cams"))
fig.add_trace(go.Scatter3d(x=eff_poses_n[:, 0].detach().cpu(),  # X coordinates
    y=eff_poses_n[:, 1].detach().cpu(),  # Y coordinates
    z=eff_poses_n[:, 2].detach().cpu(),  # Z coordinates
                           mode='markers', marker=dict(size=5, color='blue'),name="eef"))

fig.update_layout(scene=dict(
                    xaxis=dict(range=[-0.6, 0.6]),
                    yaxis=dict(range=[-0.6, 0.6]),
                    zaxis=dict(range=[0, 1.2])
))

fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                  title="Camera Positions and Orientations")

fig.update_layout(width=1000, height=700)
fig.show()